<img src="../ucsb_logo_seal.png"> 

## MLlib Classification
### PSTAT 135 / 235: Big Data Analytics
### University of California, Santa Barbara
### Last Updated: Sep 4, 2019

---  


**Sources:**  
Learning Spark, Chapter 11: Machine Learning with MLlib

https://spark.apache.org/docs/latest/mllib-linear-methods.html#logistic-regression

https://spark.apache.org/docs/latest/mllib-naive-bayes.html

http://spark.apache.org/docs/latest/mllib-decision-tree.html

http://spark.apache.org/docs/latest/mllib-ensembles.html

http://spark.apache.org/docs/latest/ml-classification-regression.html#linear-support-vector-machine

https://spark.apache.org/docs/2.1.2/api/java/org/apache/spark/mllib/util/MLUtils.html




### OBJECTIVES
- Introduce the major classification methods in MLlib
 


### CONCEPTS

- Supervised learning
- Binary and multiclass classification
- Logistic Regression
- Naive Bayes
- Tree methods
- Decision Trees
- Random Forests
- Gradient-Boosted Trees
- Ensemble
- Support vector machines

---

**Introduction to Classification**

Classification is a common form of supervised learning.  
In supervised learning, the training examples include labels.  
After training the model, the purpose of the task is to predict labels for new examples.  

The data type of the $Y$ variable makes it a *classification problem*, namely $Y$ is a discrete variable.  
Binary classification is most common. Examples include fraud (or not), default, survival, claim filing, spam.

A continuous $Y$ variable results in a regression problem (next topic).

Classification and regression both use the `LabeledPoint` class.  
To remind ourselves, a `LabeledPoint` consists of a label and a features vector.  

Follow this convention for labels:  
- For binary classification, use labels $0$ and $1$  
- For multiclass classification, use labels $0$, $1$, …, $C-1$ where $C$ is the number of classes  



Spark supports several popular models for classification including:  
- Logistic regression  
- Naive Bayes  
- Tree methods (e.g., decision tree, random forest)  
- Support Vector Machines  



**Logistic regression**    
This is currently the most popular method for binary classification.  
It is a generalized linear model which uses a linear plane to separate positive and negative examples.  
Although the model is relatively simple, the results can be very competitive.  

Below is an example of some data and a logistic curve fit to the data. Probability of Passing $Y$ is a function of Hours Studying $X$.  Notice the $Y$ variable consists of the values 0, 1.



<img src="logreg_img2.png">

MLlib includes the stochastic gradient descent (`SGD`) and `L-BFGS` algorithms for fitting the model.  
`L-BFGS` is an approximation to Newton’s method that converges faster; it is the recommended method. 

From the documentation:

“ `L-BFGS` version is strongly recommended since it converges faster and more accurately compared to `SGD` by approximating the inverse Hessian matrix using quasi-Newton method.”

Multiclass Problems  
The algorithm will output a multinomial logistic regression model, which contains $K−1$ binary logistic regression models regressed against the first class. Given a new data point, $K−1$ models will be run, and the class with largest probability will be chosen as the predicted class.

**Logistic Regression Implementation**

The names of the model fitting functions include the algorithm applied, when there is a choice.  
For logistic regression, the following functions are supported:

- `LogisticRegressionWithLBFGS`
- `LogisticRegressionWithSGD`


**Logistic Regression: load data/train model/predict**

In [ ]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel
from pyspark.mllib.regression import LabeledPoint

# Load and parse the data
def parsePoint(line):
    values = [float(x) for x in line.split(' ')]
    return LabeledPoint(values[0], values[1:])

data = sc.textFile("data/mllib/sample_svm_data.txt")
parsedData = data.map(parsePoint)

# Print a record to understand the data structure
print(parsedData.take(1))

# Build the model
model = LogisticRegressionWithLBFGS.train(parsedData)

# Evaluating the model on training data
labelsAndPreds = parsedData.map(lambda p: (p.label, model.predict(p.features)))

# Source: https://spark.apache.org/docs/latest/mllib-linear-methods.html#logistic-regression

**Naive Bayes**  

Naive Bayes (NB) is a relatively simple model, yet the performance can be quite good.  This has led to its popularity.  

NB does multiclass classification. It is commonly used in text classification where the input features are count variables.

At a high level, the count of a word on a page can adjust the probability that the page belongs to a given class.  For example, the presence of the word “tacos” will increase the probability that the page belongs to a **restaurant** relative to a **florist**.

The algorithm computes the conditional probability distribution of each feature given a label, and then it applies Bayes’ theorem to compute the conditional probability distribution of a label given an observation.

Naive?  
The term “naive” comes from the simplifying assumption of independence between every pair of features. This assumption greatly simplifies the model and is often reasonable.


**Naive Bayes Implementation`**  

Two methods are supported:  

- multinomial naive Bayes (function name: `NaiveBayes`)
- Bernoulli naive Bayes

**Parameters**  
The model type is selected with an optional parameter “multinomial” or “bernoulli” with “multinomial” as the default.  

Additive smoothing can be used by setting the parameter $λ$ (default = 1.0)  

For document classification, the input feature vectors are usually sparse, and sparse vectors should be supplied as input to take advantage of sparsity. 


**Naive Bayes Example: load data/train model/predict**

*Preamble*  

The data used in this example is in libsvm format.  
This has sparse format like this:  

`0 1:51 2:159 3:253`

`label index1:value1 index2:value2…`

The data is read using:   `MLUtils.loadLibSVMFile`

`MLUtils`  
Helper methods to load, save and pre-process data used in `MLlib`.


In [ ]:
from pyspark.mllib.classification import NaiveBayes, NaiveBayesModel
from pyspark.mllib.util import MLUtils

# Load and parse the data file. Note this data is in sparse format.
data = MLUtils.loadLibSVMFile(sc, "data/mllib/sample_libsvm_data.txt")

# Split data approximately into training (60%) and test (40%)
training, test = data.randomSplit([0.6, 0.4])

# Train a naive Bayes model.
model = NaiveBayes.train(training, 1.0)

# Make prediction and test accuracy.
labelsAndPreds = test.map(lambda p: (p.label, model.predict(p.features)))
accuracy = 1.0 * labelsAndPreds.filter(lambda pl: pl[0] == pl[1]).count() / test.count()
print('model accuracy {}'.format(accuracy))

# Source: https://spark.apache.org/docs/latest/mllib-naive-bayes.html

**Tree Methods**  

Tree methods can be used for both classification and regression  

Simplest method is a Decision Tree, which is intuitively appealing due to series of binary decisions (Male/Female, Age greater than 30 or not)  

Can handle missing values (in many implementations), categorical data, continuous data.  
Minimal preprocessing needed.  

Feature selection is part of algorithm (best feature is used, then next best, …)  

Does not require scaling  
Handles non-linear interactions  
Handles multiclass classification  

**Decision Tree Architecture**  

Tree uses binary splits on one feature at a time  
Top of tree is the root  
Paths or branches emanate from nodes  
Bottom layer of nodes called the leaf nodes, which contain predictions  

**Decision Tree Implementation**  

`mllib.tree.DecisionTree` class  
`trainClassifier()`

Implementation partitions data by rows, allowing distributed training with millions of instances

Parameters  
Node impurity measures the homogeneity of the labels in the leaf nodes.  
Two options are available for classification: Gini impurity and entropy.

**Decision Tree Example: load data/train model/predict**  

In [ ]:
from pyspark.mllib.tree import DecisionTree
from pyspark.mllib.util import MLUtils

# Load and parse the data file
data = MLUtils.loadLibSVMFile(sc, 'data/mllib/sample_libsvm_data.txt')
# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3])

# Train a DecisionTree model.
#  Empty categoricalFeaturesInfo indicates all features are continuous.
model = DecisionTree.trainClassifier(trainingData, numClasses=2, categoricalFeaturesInfo={},
                                     impurity='gini', maxDepth=5, maxBins=32)

# Evaluate model on test instances and compute test error
predictions = model.predict(testData.map(lambda x: x.features))
labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions)
testErr = labelsAndPredictions.filter(
    lambda lp: lp[0] != lp[1]).count() / float(testData.count())
print('Test Error = ' + str(testErr))

**Tree-Based Ensemble Methods**

*Ensembles* combine multiple models together to produce a new model.  
They may consist of models of the same type (e.g., all decision trees) or mixed type (e.g., decision tree + neural net + svm)  

One of the fundamental results in machine learning is that multiple weak classifiers can be combined to produce a strong classifier.  

Ensembles are useful in reducing overfitting, since predictions are based on several different trees  

The two most popular tree-based ensemble methods are *Random Forests* and *Boosted Trees* (e.g. *Gradient-Boosted Trees*)  

They are popular because they are often very competitive  

The nice properties of decision trees carry over to ensembles of trees  

This combining step can proceed using different methods, including:  

- voting (for classification)
- averaging (for regression) 
- running model predictions through another model (classification and regression)

There are downsides to ensembles:  

- Multiple models need to be trained, loaded, and maintained  
- Model explanation is harder: no p-values like regression, several trees are feeding overall decision.  
There are methods to provide feature importance information, such as partial dependence plots.

**Random Forest**  
Ensembles of decision trees  

RFs inject two sources of randomness into modeling:  

1. At each step, randomly select $p$ features out of $n$ total features for possible inclusion (random subspace method)
2. Sample the original training set with replacement, up to the size of the original training set (bootstrapping of the training set)

The number of features to randomly select $p$ is a parameter  
The number of bootstrapped trees to grow $N$ is a parameter  

Since the trees are grown independently, the training and prediction tasks are embarrassingly parallel and can be assigned to multiple workers.

Classification prediction done by majority vote across trees

**Random Forest Implementation**

`from pyspark.mllib.tree import RandomForest`  

Two most important parameters (which should be tuned using $k$-fold cross validation):  

- `numTrees`: Number of trees in forest
More trees will increase accuracy but also training time  

- `maxDepth`: Maximum depth of each tree in forest
Increasing depth can increase power of model, but will take longer to train and can overfit  

Other important parameters:

- `subsamplingRate`: fraction of size of original training set (default=1.0 recommended)

- `featureSubsetStrategy`: specified as fraction or function of total number of features

**Random Forest Example: load data/train model/predict**  
NOTE: Very similar to Decision Tree code above


In [ ]:
from pyspark.mllib.tree import RandomForest
from pyspark.mllib.util import MLUtils

# Load and parse the data file
data = MLUtils.loadLibSVMFile(sc, 'data/mllib/sample_libsvm_data.txt')
# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3])

# Train a RandomForest model.
#  Empty categoricalFeaturesInfo indicates all features are continuous.
#  Setting featureSubsetStrategy="auto" lets the algorithm choose.
model = RandomForest.trainClassifier(trainingData, numClasses=2, categoricalFeaturesInfo={},
                                     numTrees=1000, featureSubsetStrategy="auto",
                                     impurity='gini', maxDepth=5, maxBins=32)

# Evaluate model on test instances and compute test error
predictions = model.predict(testData.map(lambda x: x.features))
labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions)
testErr = labelsAndPredictions.filter(
    lambda lp: lp[0] != lp[1]).count() / float(testData.count())
print('Test Error = ' + str(testErr))

**Gradient-Boosted Trees**  

GBTs work by building a sequence of trees and combining their predictions at each iteration.  The trees constructed are generally *stumps* which use a single decision split.  A stump is an example of a weak learner.

This is different from random forests, where each tree independently gives predictions on each training instance.



A loss is specified and an optimization problem is solved whereby the objective is to minimize the loss of the model by adding weak learners using a gradient-descent-like procedure.

The procedure follows a stage-wise additive model, meaning that one new weak learner is
added at a time and existing weak learners are left unchanged.
For the original work, see:

*Friedman, Jerome H. "Greedy function approximation: a gradient boosting machine." Annals of Statistics (2001): 1189–1232.*


**Gradient-Boosted Trees Implementation**  

Since the trees are built in a sequential fashion, the algorithm can not be run in parallel.  
However, shallow trees (e.g., stumps) can be used effectively; this saves time versus random forests, which use deeper trees.

The loss function in classification problems is the log loss, equal to twice the binomial negative log likelihood.

Important parameters:
- `numIterations`:  equal to the number of trees in the ensemble.  More trees means longer runtime but also better performance up to a point.
- `learningRate`:  how quickly the model adapts on each iteration. A smaller value may help the algo have better performance, but at the cost of additional runtime. The documentation recommends NOT tuning this param.

The method `runWithValidation` can help mitigate overfitting.  It takes a training RDD and a validation RDD.

The training is stopped when the improvement in the validation error is not more than a certain tolerance (supplied by the `validationTol` argument in `BoostingStrategy`).

**GBT Example: load data/train model/predict**

In [ ]:
from pyspark.mllib.tree import GradientBoostedTrees
from pyspark.mllib.util import MLUtils

# Load and parse the data file.
data = MLUtils.loadLibSVMFile(sc, "data/mllib/sample_libsvm_data.txt")
# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3])

# Train a GradientBoostedTrees model.
model = GradientBoostedTrees.trainClassifier(trainingData,
                                             categoricalFeaturesInfo={}, numIterations=1000)

# Evaluate model on test instances and compute test error
predictions = model.predict(testData.map(lambda x: x.features))
labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions)
testErr = labelsAndPredictions.filter(
    lambda lp: lp[0] != lp[1]).count() / float(testData.count())
print('Test Error = ' + str(testErr))

**Linear Support Vector Machine**  

An SVM constructs a hyperplane or set of hyperplanes in a high- or infinite-dimensional space.  

Can be used for classification, regression, or other tasks.   
Intuitively, a good separation is achieved by the hyperplane that has the largest distance to the nearest training-data points (*support vectors*).

**Linear Separation using SVM in the Classification Task**

<img src="svm_img2.png">

**SVM Implementation**  

Internally, it optimizes the `Hinge Loss` using OWLQN optimizer.  

Important Parameters
- `regParam`  regularization parameter

**SVM Example: load data/train model/predict**

In [ ]:
from pyspark.ml.classification import LinearSVC

# Load training data
training = spark.read.format("libsvm").load("data/mllib/sample_libsvm_data.txt")

lsvc = LinearSVC(maxIter=10, regParam=0.1)

# Fit the model
lsvcModel = lsvc.fit(training)

# Print the coefficients and intercept for linearsSVC
print("Coefficients: " + str(lsvcModel.coefficients))
print("Intercept: " + str(lsvcModel.intercept))